In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy
import scipy
import matplotlib
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import pytz
import multiprocessing as mpp
import pickle
import os
import time
import numba
#
import pyspark
import pyspark.sql.types as ps_types
import h5py
#
# TODO: phase out unreferenced hpc_lib calls...
import hpc_lib

#
#data_file_name = 'data/mazama_usage_20200506_tool8.out'
#data_file_name = 'data/sacct_sherlock_out_serc2020_05_08.out'

#data_file_name = 'data/serc_usage_20200914.out'
data_file_name = 'data/sacct_mazama_20200820.out'
data_file_name = 'data/sacct_owners_out_3500489.out'
#
pkl_name = "{}.pkl".format(os.path.splitext(data_file_name)[0])
h5_name = "{}.h5".format(os.path.splitext(data_file_name)[0])
#

### PySpark implementation of HPC_analytics (prototype)
- Right now, a hybrid of vanilla `RDD` and `PySpark DataFrames`
- Mixing a bit of using `SQLContext` context methods and whatever the native (??) Spark context.

#### Brief sumary:
PySpark is the Python implementation of Spark, which is a distributed data processing infrastructure. Spark should parallelize across multiple nodes, and so should be a better multi-processing option than `python.multiprocessing`. The syntax is not Pythonic. At all, so it's basically like writing another language in Python, and the workflow strategies are very different as well, so if you're a Python person, be prepared to pivot a bit.

The biggest problem I'm still having is the final act of reading a very large data file into memory or transfering into a new (disk based) container. This, of course, should be simple, since it is really the fundamental and primary purpose of Spark, but alas... The problem arises when the distributed (Java) VMs exceed memory limitations. But we'll get there...

#### 1. A quick look at the inpuit data:

In [3]:
# Take a quick look at the input data:
#
with open(data_file_name, 'r') as fin:
    k=0
    for rw in fin:
        print('** ', rw)
        k+=1
        if k>10: break

**  User|Group|GID|JobName|JobID|JobIDRaw|Partition|State|Timelimit|NCPUS|NNodes|Submit|Eligible|Start|End|Elapsed|SystemCPU|UserCPU|TotalCPU|NTasks|CPUTimeRAW|Suspended|ReqGRES|AllocGRES|ReqTRES|AllocTRES|

**  tsy935|rubin|9133|dshap|57307350|57307350|owners|COMPLETED|2-00:00:00|12|1|2020-01-01T00:05:05|2020-01-01T00:07:06|2020-01-01T01:14:30|2020-01-01T17:18:46|16:04:16|2-21:34:59|2-16:33:26|5-14:08:26||694272|00:00:00|||billing=12,cpu=12,mem=192G,node=1|billing=12,cpu=12,energy=7143296,mem=192G,node=1|

**  |||batch|57307350.batch|57307350.batch||COMPLETED||12|1|2020-01-01T01:14:30|2020-01-01T01:14:30|2020-01-01T01:14:30|2020-01-01T17:18:46|16:04:16|2-21:34:59|2-16:33:26|5-14:08:26|1|694272|00:00:00||||cpu=12,mem=192G,node=1|

**  |||extern|57307350.extern|57307350.extern||COMPLETED||12|1|2020-01-01T01:14:30|2020-01-01T01:14:30|2020-01-01T01:14:30|2020-01-01T17:18:46|16:04:16|00:00.001|00:00:00|00:00.001|1|694272|00:00:00||||billing=12,cpu=12,mem=192G,node=1|

**  tsy935|rubin|9133

#### 2. Instantiate and configure some context handler objects.
- There are a few...
- The "spark" and "sql" variants seem to come from different branches of the project, or source projects, that have since merged, albeit perhaps not entirely gracefully.

### Build some working components:
- Spark context objects
- Type dictionary translator
- Other type, etc. translator maps


In [4]:
n_cpu = 6
#
# .config("spark.driver.memory", "15g")
#conf = pyspark.SparkConf('local[*]').set("spark.cores.max", "6").set("spark.executor.instances", "4").set("spark.executor.cores","2")
conf = pyspark.SparkConf('local[{}]'.format(n_cpu)).set("spark.driver.memory", "16g")
#
#conf = conf.set("spark.executor.memory", "4g").set("spark.executor.pyspark.memory", "3g")
sc   = pyspark.SparkContext(conf=conf)
#

# also build a SQL context? We never ended up using this, though I think you can use
#. it to do SQL querries on the data sets.
sc_sql = pyspark.SQLContext(sc)
#
spark = pyspark.sql.SparkSession.builder.appName('HPC_loader').master('local[{}]'.format(n_cpu)).config("spark.driver.memory", "15g").getOrCreate()
#spark = pyspark.sql.SparkSession.builder.appName('HPC_loader').config(conf).getOrCreate()
#
#sc.stop()
#spark.stop()

Pull some data structures, handler functions, etc. from relevant modules (ie `hpc_lib`). Note that eventually, we'll want to consolicate the `process_row()` function.

In [5]:
delim = '|'
types_dict = hpc_lib.SACCT_data_handler.default_types_dict
print('** typex_dict: ', types_dict)
#
# some numpy. (and other?) data types are not (well) supported by Spark, most pointedly some of the 
#. numpy.float{k} variants, so write a dictionary/map to re-type them. NOTE: this might need to include
#. some numpy.int{k} types as well.
numpy_re_typer={numpy.float64:float, numpy.float128:float, numpy.int64:int }
#
# python functions to use on grouped/aggregated columns:
group_py_functions = {'End':numpy.nanmax, 'Start':numpy.nanmin, 'NCPUS':numpy.nanmax, 'NNodes':numpy.nanmax}
#
# Define one or more row processing functions.
def f_rw(rw, header_names, RH_index, types_dict=types_dict, delim='|'):
    #rws = rw[:-1].split(delim)
    rws = rw.split(delim)
    #
    #if not len(rws)==0:
    #    return rws
    return [None if s=='' else types_dict.get(h,str)(s) for h,s in zip(header_names,rws)] + [rws[RH_index['JobID']].split('.')[0] ]
    #return [str(s) for h,s in zip(header_names,rws)]
#
# this will be used to reduce() the raw data to summary data:
def f_reduce_row(r1, r2, header_names, verbose=False):
    if verbose:
        print('*** DEBUG: lens: ', len(r1), len(r2), len(header_names))
    return tuple([group_py_functions.get(hdr, lambda x: x[0] )([x1, x2]) 
             for k, (hdr,x1,x2) in enumerate(zip(header_names,
                                *sorted([r1, r2], key=lambda X:X[RH_index['Submit']]))) ] )

#
def spark_types_to_numpy(spark_df):
    # row1 = df1.agg({"x": "max"}).collect()[0]
    #
    # peel off the dtypes so it will pickle properly:
    spark_dtypes = spark_df.dtypes
    #
    lens = spark_df.rdd.map(lambda rw: [max(1, len( (s or '') ) ) if spark_dtypes[k][1]=='string'
                                else None for k,s in enumerate(rw) ] ).reduce(lambda x1,x2: max(x1,x2))
    #
    #print('*** stn: lens:: ', lens)
    #
    dtypes_out = []
    for (nm,tp), s_len in zip(spark_dtypes, lens):
        #print('*** DEBUG:: dtype[{},{}]: {}'.format(nm,tp,s_len))
        if tp in ('int', 'bigint'):
            tp_n = '>i8'
            #tp_n = '>f8'
            # do we need a casting function?
            #nm_f = float
        elif tp in ('float', 'double'):
            tp_n = '>f8'
        elif tp in ('string'):
            tp_n = 'S{}'.format(s_len)
        #
        dtypes_out += [(nm, tp_n)]
    #
    return dtypes_out

# numpy_to_spark_types = {str:'StringType'}


** typex_dict:  {'User': <class 'str'>, 'JobID': <class 'str'>, 'JobName': <class 'str'>, 'Partition': <class 'str'>, 'State': <class 'str'>, 'JobID_parent': <class 'str'>, 'Timelimit': <function elapsed_time_2_day at 0x7fd091b2f700>, 'Start': <function str2date_num at 0x7fd091b2f5e0>, 'End': <function str2date_num at 0x7fd091b2f5e0>, 'Submit': <function str2date_num at 0x7fd091b2f5e0>, 'Eligible': <function str2date_num at 0x7fd091b2f5e0>, 'Elapsed': <function elapsed_time_2_day at 0x7fd091b2f700>, 'MaxRSS': <class 'str'>, 'MaxVMSize': <class 'str'>, 'NNodes': <class 'int'>, 'NCPUS': <class 'int'>, 'MinCPU': <class 'str'>, 'SystemCPU': <function elapsed_time_2_day at 0x7fd091b2f700>, 'UserCPU': <function elapsed_time_2_day at 0x7fd091b2f700>, 'TotalCPU': <function elapsed_time_2_day at 0x7fd091b2f700>, 'NTasks': <class 'int'>}


In [6]:
# Preliminarily, read file into line-strings. Extract header row so that we
#. can type() each column when we split() it up.
#
lines = sc.textFile(data_file_name)
# header_names = (lines.take(1)[0])[:-1].split(delim)
# RH_index = {s:k for k,s in enumerate(header_names) }
# #
# print('** Headers: ', header_names)

### Excluding header row from data collection:
- This is surprisingly harder than it looks, and searching for solutions seems to be elusive.
-  I think the most direct and common approach is to read text data via `textFile()`, then use filter() to exclude rows that look like the first row. This sounds kinda dumb for a single file, but it is a good option when reading multiple files.
- *(spark) dataframes:* Use a syntactical variant of `spark.read.format('CSV')` method below to read the data into a dataframe. This provides options to exclude and catch the header row.


#### Dataframes:
- using the sql context, we can read the data into a dataframe
- Nominally fast and easy, but I think really for well behaved data.
- Getting the header row is not too tough, but I'm not so sure about excluding a false terminal column, resulting from a terminal delimeter (row string ending in a delimiter).
- In fact, we seem to get some weird behavior from this
- ... to the point that I would probably just err on the side of having more control and maybe burning some cycles on the filter() option (which i expect is pretty well optimized on the back end).
- HOWEVER: Preliminary assessments just doing a `.count()` suggests that DF might be much, much faster than the standard RDD methods... Though that may also be because the `DataFramds` methods are using a context or session that is not CPU limited -- which would make sense.


In [7]:
# # Another way to read the file with headers. This will give an effective array of (val,ky) tuples.
# df_rows1 = spark.read.format('CSV').option('header', 'true').option('sep', '|').load(data_file_name)
# #
# # another syntax:
# df_rows = spark.read.csv(data_file_name, header=True, sep='|')
# #

# for rw in df_rows1.take(10):
#     print('** ', rw[:])

# # Another syntax to load directly into a spark dataframe (via .sql):
# #
# #
# print('** type: ', type(rows_2))
# print('** type: ', type(df_rows))
# print('** dypes: ', df_rows.dtypes)
# print('** header: {}'.format( df_rows.schema.names ) )
# #
# print('\n*** *** ')
# for rw in df_rows.take(5):
#     print('** ', rw[:])
#     #print('* * ', rw.head)
#
# print('*** ', df_rows.schema)
# #
# print('*** ', df_rows.dtypes)
# #print('** ', set())

In [8]:
#
# identifying the header row(s) and filtering bogus terminal characters on each row:
n_terminal = 0
header_string = lines.take(1)[0]
#
while header_string[-1] in ('\n', delim):
    header_string=header_string[:-1]
    n_terminal += 1
#
# if necessary, trim off row-terminal characters:
print('** n_terminal: ', n_terminal)
if n_terminal>0:
    lines = lines.map(lambda ln: ln[:-n_terminal])
#
header_names = header_string.split(delim) + ['JobID_parent']
RH_index = {s:k for k,s in enumerate(header_names) }
print('** Headers[{}]: {}'.format(len(header_names), header_names) )
#
# for c in (lines.take(2)[1]):
#     print('{}: [{}]'.format(c, ord(c)))

** n_terminal:  1
** Headers[27]: ['User', 'Group', 'GID', 'JobName', 'JobID', 'JobIDRaw', 'Partition', 'State', 'Timelimit', 'NCPUS', 'NNodes', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed', 'SystemCPU', 'UserCPU', 'TotalCPU', 'NTasks', 'CPUTimeRAW', 'Suspended', 'ReqGRES', 'AllocGRES', 'ReqTRES', 'AllocTRES', 'JobID_parent']


In [9]:
# use this to filter header rows:
n_startswith = 15
header_start = header_string[0:n_startswith]
#
rows = lines.filter(lambda s: not s.startswith(header_start) ).map(lambda x: f_rw(x, header_names,
                                            types_dict=types_dict, RH_index=RH_index) )
rows = rows.filter(lambda rw:rw[RH_index['JobName']]!='batch' and rw[RH_index['JobName']]!='extern' ).map(lambda rw: [None if (x is None or x=='') else numpy_re_typer.get(type(x), type(x))(x) for x in rw])
#
end_max = rows.map(lambda rw: rw[RH_index['End']]).filter(lambda x: x is not None).max()
print('** end_max: ', end_max)

** end_max:  737608.6457986111


In [10]:
rows = rows.map(lambda rw: 
                rw[0:RH_index['End']] + [end_max] + rw[RH_index['End']+1:] 
                if rw[RH_index['End']] is None else rw  )
print('** ', rows.take(10))

**  [['tsy935', 'rubin', '9133', 'dshap', '57307350', '57307350', 'owners', 'COMPLETED', 2.0, 12, 1, 737425.0035300925, 737425.0049305556, 737425.051736111, 737425.7213657408, 0.6696296296296296, 2.8992939814814815, 2.689884259259259, 5.589189814814815, None, '694272', '00:00:00', None, None, 'billing=12,cpu=12,mem=192G,node=1', 'billing=12,cpu=12,energy=7143296,mem=192G,node=1', '57307350'], ['tsy935', 'rubin', '9133', 'dshap', '57274801', '57274801', 'owners', 'TIMEOUT', 2.0, 12, 1, 737425.0317476852, 737425.0331481481, 737425.1057638889, 737427.1060185186, 2.0002546296296297, 6.843449074074074, 6.841064814814815, 13.684513888888889, None, '2073864', '00:00:00', None, None, 'billing=12,cpu=12,mem=192G,node=1', 'billing=12,cpu=12,mem=192G,node=1', '57274801'], ['xiankong', 'jianq', '282061', 'li', '57395093', '57395093', 'owners', 'FAILED', 2.0, 20, 1, 737425.032962963, 737425.0343634259, 737425.0356597222, 737425.0809027777, 0.04524305555555556, 0.08324074074074074, 0.820497685185185

In [11]:
print('** lens: ', len(rows.take(1)[0]), len(header_names))
print('** header_names: ' , [(k,h) for k,h in enumerate(header_names) ])
print('** row1: ', [(k,c) for k,c in enumerate(rows.take(1)[0])])
l1,l2 = lines.take(2)
print('** lines hdr: ', l1)
print('** lines1: ', l2)
print('** ', len(lines.take(1)[0].split('|')) )
print('** ', len(lines.take(2)[1].split('|')) )
#
# we can either group and reduce using the RH{} index, or convert to a DF first:
#rows_df = spark.createDataFrame( rows, header_names ).sort('JobID')
#
# print('** rows_df schema[{}]: {}'.format(len(rows_df.schema), rows_df.schema))
# print('** dytpes[{}]: {}'.format(len(rows_df.dtypes), rows_df.dtypes))
# print('** header_names[{}]: {}'.format(len(header_names), header_names))

** lens:  27 27
** header_names:  [(0, 'User'), (1, 'Group'), (2, 'GID'), (3, 'JobName'), (4, 'JobID'), (5, 'JobIDRaw'), (6, 'Partition'), (7, 'State'), (8, 'Timelimit'), (9, 'NCPUS'), (10, 'NNodes'), (11, 'Submit'), (12, 'Eligible'), (13, 'Start'), (14, 'End'), (15, 'Elapsed'), (16, 'SystemCPU'), (17, 'UserCPU'), (18, 'TotalCPU'), (19, 'NTasks'), (20, 'CPUTimeRAW'), (21, 'Suspended'), (22, 'ReqGRES'), (23, 'AllocGRES'), (24, 'ReqTRES'), (25, 'AllocTRES'), (26, 'JobID_parent')]
** row1:  [(0, 'tsy935'), (1, 'rubin'), (2, '9133'), (3, 'dshap'), (4, '57307350'), (5, '57307350'), (6, 'owners'), (7, 'COMPLETED'), (8, 2.0), (9, 12), (10, 1), (11, 737425.0035300925), (12, 737425.0049305556), (13, 737425.051736111), (14, 737425.7213657408), (15, 0.6696296296296296), (16, 2.8992939814814815), (17, 2.689884259259259), (18, 5.589189814814815), (19, None), (20, '694272'), (21, '00:00:00'), (22, None), (23, None), (24, 'billing=12,cpu=12,mem=192G,node=1'), (25, 'billing=12,cpu=12,energy=7143296,me

In [12]:
# Here, let's try a RDD_pair, then reduce_by_key() function:
# (this appears to work, but needs to be validated )
row_pairs = rows.map(lambda x: (x[RH_index['JobID_parent']], list(x[:])))
#
summary_rdd = row_pairs.reduceByKey(lambda rw1, rw2: f_reduce_row(rw1, rw2, header_names))
#

for rw in rows.take(10):
    print('** ', rw)
#
print('reduced: ')
for rw in summary_rdd.take(10):
    print('** ', rw)

**  ['tsy935', 'rubin', '9133', 'dshap', '57307350', '57307350', 'owners', 'COMPLETED', 2.0, 12, 1, 737425.0035300925, 737425.0049305556, 737425.051736111, 737425.7213657408, 0.6696296296296296, 2.8992939814814815, 2.689884259259259, 5.589189814814815, None, '694272', '00:00:00', None, None, 'billing=12,cpu=12,mem=192G,node=1', 'billing=12,cpu=12,energy=7143296,mem=192G,node=1', '57307350']
**  ['tsy935', 'rubin', '9133', 'dshap', '57274801', '57274801', 'owners', 'TIMEOUT', 2.0, 12, 1, 737425.0317476852, 737425.0331481481, 737425.1057638889, 737427.1060185186, 2.0002546296296297, 6.843449074074074, 6.841064814814815, 13.684513888888889, None, '2073864', '00:00:00', None, None, 'billing=12,cpu=12,mem=192G,node=1', 'billing=12,cpu=12,mem=192G,node=1', '57274801']
**  ['xiankong', 'jianq', '282061', 'li', '57395093', '57395093', 'owners', 'FAILED', 2.0, 20, 1, 737425.032962963, 737425.0343634259, 737425.0356597222, 737425.0809027777, 0.04524305555555556, 0.08324074074074074, 0.8204976851

In [13]:
# r1, r2 = rows.take(2)
# print('** ', r1)
# print('** ', r2)
# #
# rr = f_reduce_row(r1, r2, header_names, True)
# print('** ** ', rr)


# N=1000
# #rows_df = spark.createDataFrame( rows.takeSample(withReplacement=True, num=N), header_names )
# rows_df = spark.createDataFrame( rows.take(num=N), header_names )

# typex_dict:  {'User': <class 'str'>, 'JobID': <class 'str'>, 'JobName': <class 'str'>, 
#               'Partition': <class 'str'>, 'State': <class 'str'>, 'JobID_parent': <class 'str'>, 
#               'Timelimit': <function elapsed_time_2_day at 0x7f035ef83790>, 
#               'Start': <function str2date_num at 0x7f035ef83670>, 
#               'End': <function str2date_num at 0x7f035ef83670>, 
#               'Submit': <function str2date_num at 0x7f035ef83670>, 
#               'Eligible': <function str2date_num at 0x7f035ef83670>, 
#               'Elapsed': <function elapsed_time_2_day at 0x7f035ef83790>, 
#               'MaxRSS': <class 'str'>, 'MaxVMSize': <class 'str'>, 'NNodes': <class 'int'>, 
#               'NCPUS': <class 'int'>, 'MinCPU': <class 'str'>, 
#               'SystemCPU': <function elapsed_time_2_day at 0x7f035ef83790>, 
#               'UserCPU': <function elapsed_time_2_day at 0x7f035ef83790>, 
#               'TotalCPU': <function elapsed_time_2_day at 0x7f035ef83790>, 'NTasks': <class 'int'>}

In [14]:
#
# make a sampling DataFrame() to automagically determine the schema (this breaks when we do just 
#. the summary... which suggests maybe a problem -- NTasks is not populating correctly maybe?)
# otherwise, we need to write some more translator dictionaries to map numpy types to Spark types
#
# for n in range(2,7):
#     N=int(10**n)
#     try:
#         rows_df = spark.createDataFrame( rows.take(N), header_names )
#         break
#     except:
#         # try a bigger sample.
#         print('broke for [{}]. trying a bigger sample.'.format(N))
#
#sp_string='string'
sp_string = ps_types.StringType
#sp_float  = ps_types.FloatType()
sp_float  = ps_types.DoubleType
#sp_int    = ps_types.IntegerType()
sp_int    = ps_types.LongType
#
spark_data_types={'User':sp_string, 'Group':sp_string, 'GID':sp_string, 'JobName':sp_string, 'JobID':sp_string,
              'JobIDRaw':sp_string, 'Partition':sp_string, 'State':sp_string, 'Timelimit':sp_float,
              'NCPUS':sp_int, 'NNodes':sp_int, 'Submit':sp_float, 'Eligible':sp_float, 'Start':sp_float,
              'End':sp_float, 'Elapsed':sp_float, 'SystemCPU':sp_float, 
              'UserCPU':sp_float, 'TotalCPU':sp_float, 'NTasks':sp_int, 'CPUTimeRAW':sp_string,
              'Suspended':sp_string, 'ReqGRES':sp_string, 'AllocGRES':sp_string, 'JobID_parent':sp_string}
#
#df_schema = [(col, spark_dtypes.get(col,  sp_string)) for col in header_names]
df_schema = ps_types.StructType([ps_types.StructField(col, spark_data_types.get(col,  sp_string)() ) 
                                 for col in header_names])

print('** ', df_schema)

# rows_df = spark.createDataFrame( rows, header_names ).sort('JobID')
summary_df = spark.createDataFrame(summary_rdd.values().map(lambda rw: 
                            [None if (x is None or x=='') else numpy_re_typer.get(type(x),
                                lambda a:a)(x) for x in rw]),
                                   df_schema)
#
for rw in summary_rdd.take(10):
    print('** ', rw)

**  StructType(List(StructField(User,StringType,true),StructField(Group,StringType,true),StructField(GID,StringType,true),StructField(JobName,StringType,true),StructField(JobID,StringType,true),StructField(JobIDRaw,StringType,true),StructField(Partition,StringType,true),StructField(State,StringType,true),StructField(Timelimit,DoubleType,true),StructField(NCPUS,LongType,true),StructField(NNodes,LongType,true),StructField(Submit,DoubleType,true),StructField(Eligible,DoubleType,true),StructField(Start,DoubleType,true),StructField(End,DoubleType,true),StructField(Elapsed,DoubleType,true),StructField(SystemCPU,DoubleType,true),StructField(UserCPU,DoubleType,true),StructField(TotalCPU,DoubleType,true),StructField(NTasks,LongType,true),StructField(CPUTimeRAW,StringType,true),StructField(Suspended,StringType,true),StructField(ReqGRES,StringType,true),StructField(AllocGRES,StringType,true),StructField(ReqTRES,StringType,true),StructField(AllocTRES,StringType,true),StructField(JobID_parent,Strin

In [15]:
# for rw in summary_df.take(10):
#     print('** ', rw[:])

# dt_dict = dict(summary_df.dtypes)
# sdf_dtypes = summary_df.dtypes
# #
# print('** ', sdf_dtypes)
# #lens = summary_df.rdd.map(lambda rw: [len(s) for k,s in enumerate(rw) if sdf_dtypes[k][1]=='string'] ).reduce(lambda x1,x2: max(x1,x2))
# lens = summary_df.rdd.map(lambda rw: [len(s) if sdf_dtypes[k][1]=='string' else None for k,s in enumerate(rw) ] ).reduce(lambda x1,x2: max(x1,x2))
# print('** lens: ', lens)



# # for rw in lens.take(10):
# #     print('** ', rw)
# print('** ')
# for (cl,n),l in zip(summary_df.dtypes, lens):
#     print('** ', cl,n,l)

In [16]:
print('** ', summary_df.dtypes)
#spark_to_numpy_types('double':float, 'float':float, 'bigint':int, 'int':int, 'string':str)
#
print('** ', header_names)
#
print('** ')
for rw in summary_df.take(10):
    print('** ', rw)

**  [('User', 'string'), ('Group', 'string'), ('GID', 'string'), ('JobName', 'string'), ('JobID', 'string'), ('JobIDRaw', 'string'), ('Partition', 'string'), ('State', 'string'), ('Timelimit', 'double'), ('NCPUS', 'bigint'), ('NNodes', 'bigint'), ('Submit', 'double'), ('Eligible', 'double'), ('Start', 'double'), ('End', 'double'), ('Elapsed', 'double'), ('SystemCPU', 'double'), ('UserCPU', 'double'), ('TotalCPU', 'double'), ('NTasks', 'bigint'), ('CPUTimeRAW', 'string'), ('Suspended', 'string'), ('ReqGRES', 'string'), ('AllocGRES', 'string'), ('ReqTRES', 'string'), ('AllocTRES', 'string'), ('JobID_parent', 'string')]
**  ['User', 'Group', 'GID', 'JobName', 'JobID', 'JobIDRaw', 'Partition', 'State', 'Timelimit', 'NCPUS', 'NNodes', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed', 'SystemCPU', 'UserCPU', 'TotalCPU', 'NTasks', 'CPUTimeRAW', 'Suspended', 'ReqGRES', 'AllocGRES', 'ReqTRES', 'AllocTRES', 'JobID_parent']
** 
**  Row(User='pjwomble', Group='gorelick', GID='26961', JobName='21c36

In [17]:
#
numpy_dtypes = spark_types_to_numpy(summary_df)

In [18]:
array_len = summary_df.count()
#
print('** numpy_dfs: ', numpy_dtypes)

** numpy_dfs:  [('User', 'S8'), ('Group', 'S8'), ('GID', 'S6'), ('JobName', 'S39'), ('JobID', 'S8'), ('JobIDRaw', 'S8'), ('Partition', 'S6'), ('State', 'S9'), ('Timelimit', '>f8'), ('NCPUS', '>i8'), ('NNodes', '>i8'), ('Submit', '>f8'), ('Eligible', '>f8'), ('Start', '>f8'), ('End', '>f8'), ('Elapsed', '>f8'), ('SystemCPU', '>f8'), ('UserCPU', '>f8'), ('TotalCPU', '>f8'), ('NTasks', '>i8'), ('CPUTimeRAW', 'S3'), ('Suspended', 'S8'), ('ReqGRES', 'S1'), ('AllocGRES', 'S1'), ('ReqTRES', 'S30'), ('AllocTRES', 'S43'), ('JobID_parent', 'S8')]


In [19]:

my_types = {'i8':'>f8', 'i16':'>f16'}
foutname = 'data/serc_spark_summary.h5'
#
#os.remove(foutname)
with h5py.File(foutname, 'w') as fout:
    ds = fout.create_dataset('summary', (array_len, ),
                            dtype=numpy_dtypes)
    #
    #ds[...] = numpy.array(summary_df.collect())[:]
    for cl,tp in numpy_dtypes:
        print('** col: {} :: {}'.format(cl, tp))
        #if not cl=='NTasks': continue
#        ds[cl] = numpy.array([numpy.nan if x is None else x for x in numpy.reshape(summary_df.select(cl).collect(),
#                                                                     (array_len,))])
        # None types for integer types are badly handled by numpy and HDF5; we have to convert them \
        #   basically manually
        if tp in ('>i8', '>i16'):
            ds[cl] = numpy.array([numpy.nan if x is None else x for x in numpy.reshape(summary_df.select(cl).collect(),
                                                                     (array_len,))])[:]
        else:
            ds[cl] = numpy.reshape(summary_df.select(cl).collect(), (array_len,)).astype(tp)[:] 
            #ds[cl] = numpy.array(numpy.reshape(summary_df.select(cl).collect(), (array_len,)), dtype=tp)


#


** col: User :: S8
** col: Group :: S8
** col: GID :: S6
** col: JobName :: S39
** col: JobID :: S8
** col: JobIDRaw :: S8
** col: Partition :: S6
** col: State :: S9
** col: Timelimit :: >f8
** col: NCPUS :: >i8
** col: NNodes :: >i8
** col: Submit :: >f8
** col: Eligible :: >f8
** col: Start :: >f8
** col: End :: >f8
** col: Elapsed :: >f8
** col: SystemCPU :: >f8
** col: UserCPU :: >f8
** col: TotalCPU :: >f8
** col: NTasks :: >i8
** col: CPUTimeRAW :: S3
** col: Suspended :: S8
** col: ReqGRES :: S1
** col: AllocGRES :: S1
** col: ReqTRES :: S30
** col: AllocTRES :: S43
** col: JobID_parent :: S8


In [20]:
with h5py.File(foutname, 'r') as fin:
    #print('NTasks:')
    for cl in ['NTasks', 'JobID', 'NCPUS', 'Elapsed']:
        print('** {}'.format(cl))
        print(fin['summary'][cl][0:20])
        

** NTasks
[-9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808]
** JobID
[b'57399044' b'57399898' b'57399898' b'57399898' b'57399898' b'57399898'
 b'57400920' b'57400920' b'57401745' b'57401745' b'57401745' b'57401745'
 b'57402775' b'57403173' b'57403223' b'57403323' b'57404945' b'57404947'
 b'57404965' b'57406142']
** NCPUS
[ 1  1  1  1  1  1  1  1  1  1  1  1 16 16 16 16 16 16 16  7]
** Elapsed
[0.01131944 0.01153935 0.01144676 0.01165509 0.01168981 0.01233796
 0.01170139 0.01152778 0.01190972 0.01180556 0.01152778 0.01033565
 0.01005787 0.02511574 0.02111111 0.02663194 0.01047454 0.01123843
 0.01094907 0.00012731]


In [21]:
# #col = 'NCPUS'
# col = 'NTasks'
# #
# with h5py.File('summary.h5', 'a') as fin:
#     print('** ', fin['summary'][col][0:10])
#     #
#     print(fin['summary']['NTasks'][0], fin['summary']['NTasks'][0]+1)
#     #fin['summary']['NTasks'][0:5] = numpy.array([7 for _ in range(5)][:])
#     #fin['summary']['NTasks'][0:5] = (numpy.ones(5)*7)[:]
#     #fin['summary']['NTasks'][0]=42.
#     fin['summary'][col,0:5] = numpy.ones(5)*43
#     fin['summary'][col,5:10] = numpy.array([numpy.nan for _ in range(5)])[:]
#     fin['summary'][col,5:10] = numpy.array([7 for _ in range(5)])[:]
#     #
#     # Not sure write_direct() will work with named columns.
#     #fin['summary'].write_direct(numpy.ones(5)*42., source_sel=numpy.s_[0:5], dest_sel=numpy.s_[5:10, RH_index[col]])
#     #
#     #fin['summary'].write_direct(numpy.ones(5, dtype='>i8')[:]*42, source_sel=numpy.s_[0:5], 
#     #                            dest_sel=numpy.s_[0:5] )
    
#     #
#     print('** ', fin['summary'][col][0:15])


In [22]:
# Let's do some validation...
#
#summary_sample = summary_df.take(100)


rows_sample = rows.take(500)

j_ids  = numpy.unique([rw[-1] for rw in rows_sample])
#
print('** jids: ', j_ids[0:10])

summary_sample = summary_df.filter(summary_df.JobID_parent.isin(list(j_ids)) ).collect()

** jids:  ['57225527' '57225616' '57225628' '57225637' '57225639' '57274400'
 '57274801' '57307118' '57307228' '57307243']


In [23]:
# print('** summary[{}]'.format(len(summary_sample)))
# for rw in summary_sample[0:10]:
#     print('** ', rw[:])
# #
# print('*** ***\n*** ***')
# #
# print('** rows[{}]: '.format(len(rows_sample)))
# for rw in rows_sample[0:10]:
#     print('** ', rw)

In [24]:
print('\n**** **** **** \nheaders[{}]: {}\n'.format(jobid, header_names) )
#jobid = '62339523'
for rw in summary_sample:
    jobid = rw[-1]
    #
    for rw in summary_sample:
        if rw[-1]==jobid:
            print('** sample: ', rw[:])
        #
    #
    print('*** ***\n\n*** ***')
    #
    k=0
    for rw in rows_sample:
        if rw[-1]==jobid and not (rw[3] in ('batch', 'extern') ):
            print('** row[{}]: {}'.format(k, rw))
            k+=1
        #
    #
    print('**\n\n')

NameError: name 'jobid' is not defined

In [ ]:
oneill_summ = summary_df.filter(summary_df.Group == 'oneillm').count()

In [ ]:
print('** ', oneill_summ )

In [ ]:
#summary_counts = row_pairs.groupByKey().filter(lambda rw:rw[3]!='extern').filter(lambda rw: rw[3]!='batch').count()
row_counts = rows.filter(lambda rw:rw[3]!='extern').filter(lambda rw: rw[3]!='batch')\
.groupBy(lambda rw:rw[-1]).countByValue()
#print('** ', row_counts.take(100))

# .agg(psf.collect_list('JobID_parent'))

In [ ]:
for k,(jid,n) in enumerate(row_counts.items()):
    if n<=1:
        continue
    #
    print('** [{}] {}: {}'.format(k,jid,n) )
#print('** ', numpy.array([(k,n) for k,(jid,n) 
#                          in enumerate(row_counts.items())])[numpy.array(row_counts.values())>1])
#print('** ', row_counts.values())